### imports

In [1]:
import pandas as pd
import random
import glob
import os

from statistics import mean
import ipywidgets as widgets

### constants

In [2]:
SCALING_FACTOR = 0.1

In [3]:
path = os.getcwd()
csv_files = sorted(glob.glob(os.path.join(path, "scores/2023/*.csv")))

In [4]:
# set up sse for each course and layout
# should we pull more data from dgcr? maybe course length per layout, etc?
# would allow us to get course names and such... that's probably better...

SSE = {
    'shmo_s': 47.7,
    'shmo_l': 54.4,
    'romo_s': 45.0, # total guess
    'romo_l': 50.4,
    'spva_s': 56.6,
    'spva_l': 64.0,
    'mbog_s': 44.8,
    'mbog_l': 53.4,
    'mbno_s': 45.3,
    'mbno_l': 53.9,
    'knol_s': 42.9,
    'knol_l': 42.9,
    'r_arro': 27.5,
    'r_arr2': 55.0,
    'vira_s': 48.1,
    'vira_l': 54.9,
    'coldbk': 67.0,
    'begg_p': 50.0,
    'brew_s': 43.9,
    'brew_l': 55.0,
    'johnsn': 43.1,
    'garfld': 46.1
}

### helper functions

In [6]:
def calculate_rating(score, sse):
    if sse > 50.328725:
        scoring_interval = -0.225067 * sse + 21.3858
    else:
        scoring_interval = -0.487095 * sse + 34.5734
    return 1000 - (score-sse)*scoring_interval

In [7]:
def calculate_target_stroke_count(rating, sse):
    return (1000/rating * sse).round(2)

In [8]:
# calculate_rating(58, SSE['shmo_l'])

In [10]:
# {
#     'joe': calculate_rating(56, SSE['mbno_l']),
#     'joshua': calculate_rating(59, SSE['mbno_l']),
#     'kevin': calculate_rating(50, SSE['mbno_s']),
#     'isaac': calculate_rating(59, SSE['mbno_s']),
#     'kyle': calculate_rating(44, SSE['mbno_s']),
#     'kat': calculate_rating(50, SSE['mbno_s']),
#     'noah': calculate_rating(64, SSE['mbno_l']),
#     'chase': calculate_rating(66, SSE['mbno_s'])
# }

### concatenate all files into a dataframe

### add each round for 2023

In [12]:
df = pd.DataFrame()
for filename in csv_files:
    date = filename.split('zoo_')[1].split('.csv')[0]
    newdf = pd.read_csv(filename)
    newdf['date'] = date
#     if 'hole_24' in newdf.columns:
#         newdf['course'] = 
#     else:
#         newdf['course'] = random.choice(['shmo_s','mbog_s','romo_s'])
    df = pd.concat([df, newdf])
df = df.sort_values('date')
# df['course'] = df['date'].map(courses2022)

In [30]:
df

,division,position,name,relative_score,total_score,hole_1,hole_2,hole_3,hole_4,hole_5,...,ctp,lp,attendance,date,hole_19,hole_20,hole_21,hole_22,hole_23,hole_24
0,LONG,1,Joe Bos,8.0,62,2,3,3,5,5,...,0,0,1,2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN
1,LONG,1,Joshua Owens,8.0,62,4,2,3,4,4,...,0,0,1,2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN
2,LONG,1,Kyle P-H,8.0,62,3,3,2,4,5,...,0,0,1,2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN
3,LONG,4,Noah Mashni,15.0,69,4,3,4,4,4,...,0,0,1,2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN
4,SHORT,1,Kevin Richmond,-2.0,52,3,3,2,3,3,...,0,0,1,2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,LONG,4,Noah Mashni,16.0,70,6,4,4,4,4,...,0,0,1,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN
0,LONG,1,Joe Bos,3.0,57,5,3,4,4,2,...,1,0,1,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN
1,LONG,2,Joshua Owens,7.0,61,5,5,3,3,3,...,0,1,1,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN
2,LONG,3,Kyle P-H,10.0,64,4,5,4,4,2,...,1,1,1,2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN


### attendance and CTP/LP

In [14]:
attendance = df[['name','attendance']].groupby('name').sum()
ctplp = df[['name','ctp','lp']].groupby('name').sum()
ctplp['extra_points'] = ctplp['ctp'] + ctplp['lp']

### per-hole stats

In [15]:
hole_stats = df[['course', 'name', 'hole_1', 'hole_2', 'hole_3', 'hole_4', 'hole_5', 'hole_6', 'hole_7',
       'hole_8', 'hole_9', 'hole_10', 'hole_11', 'hole_12', 'hole_13',
       'hole_14', 'hole_15', 'hole_16', 'hole_17', 'hole_18', 'hole_19', 'hole_20', 'hole_21', 'hole_22',
       'hole_23', 'hole_24']]
for col in hole_stats.columns:
    hole_stats[col] = pd.to_numeric(hole_stats[col]) if 'hole_' in col else hole_stats[col]
hole_stats_by_course = hole_stats.groupby(['course','name']).mean()
hole_stats_by_name = hole_stats.groupby(['name','course']).mean()

/tmp/ipykernel_49749/4025216453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hole_stats[col] = pd.to_numeric(hole_stats[col]) if 'hole_' in col else hole_stats[col]


In [16]:
# hole_stats_by_name

### pivot round scores to total strokes

filling NA with course-wise average per player

In [17]:
round_strokes = df[['course','name','total_score','date']].set_index(['date','name'])
round_strokes = round_strokes.pivot_table(
    values='total_score',
    index=['date'],
    columns='name',
    aggfunc='first'
)

round_strokes['the_field'] = round_strokes.mean(axis=1)
# round_scores = round_scores.fillna(round_scores.groupby('course').transform('mean')).fillna(round_scores.mean()).round(2)
round_strokes

name,Chase Ramsey,Isaac Richmond,Joe Bos,Joshua Owens,Kat Owens,Kevin Richmond,Kyle P-H,Noah Mashni,the_field
date,,,,,,,,,
2023-04-19,NaN,70.0,62.0,62.0,57.0,52.0,62.0,69.0,62.000000
2023-04-26,64.0,70.0,60.0,64.0,54.0,59.0,67.0,64.0,62.750000
2023-05-03,65.0,71.0,62.0,62.0,60.0,55.0,59.0,60.0,61.750000
2023-05-10,NaN,60.0,53.0,57.0,59.0,55.0,60.0,73.0,59.571429
2023-05-17,66.0,59.0,56.0,59.0,50.0,50.0,44.0,64.0,56.000000
2023-05-24,100.0,91.0,69.0,75.0,86.0,81.0,79.0,85.0,83.250000
2023-05-31,63.0,62.0,55.0,62.0,57.0,59.0,53.0,54.0,58.125000
2023-06-07,63.0,63.0,62.0,51.0,64.0,49.0,56.0,59.0,58.375000
2023-06-28,NaN,NaN,54.0,58.0,52.0,NaN,62.0,63.0,57.800000


### ratings

generate round ratings per player based on SSE

In [18]:
ratings = df[['course','name','total_score','date']].set_index(['date','name'])
ratings['sse'] = ratings['course'].map(SSE)
ratings['rating'] = ratings.apply(lambda row: calculate_rating(row['total_score'],row['sse']),axis=1)
ratings = ratings.drop_duplicates()
ratings = ratings.pivot_table(
    values = 'rating',
    index = 'date',
    columns = 'name',
    aggfunc = 'first'
).drop_duplicates()
ratings['the_field'] = ratings.mean(axis=1)
# ratings

### rating average

calculate a rolling average of players' rating as calculated above

In [19]:
ratings_ave = ratings.fillna(ratings.mean()).rolling(6, axis=0, min_periods=4).apply(lambda x: mean(sorted(x)[1:4]))
# ratings_ave

#### ratings against field

unclear if this will be valuable, but easy to calc

In [20]:
# ratings_field = ratings_ave.apply(lambda x: x-ratings_ave['the_field'])
# ratings_field

### expected strokes

based on current rating as calculated above, this is the number of strokes anticipated for each player at each round. factors into handicap and therefor ranking points.

In [21]:
## deprecated

# expected_strokes = ratings_ave.copy()
# expected_strokes['course'] = ratings['course']
# expected_strokes['sse'] = ratings['sse']
# for player in PLAYERS:
#     expected_strokes[player] = expected_strokes.apply(lambda x:(1000/x[player] * x['sse']), axis=1)
# expected_strokes

### ranking points

this is determined by adjusting scores by individual handicap and ranking those adjusted scores. the number of those bested is the number of points granted.

In [22]:
diff = ratings - ratings_ave
ranked = diff.drop(columns=['the_field']).rank(axis=1,ascending=True)
ranked

name,Chase Ramsey,Isaac Richmond,Joe Bos,Joshua Owens,Kat Owens,Kevin Richmond,Kyle P-H,Noah Mashni
date,,,,,,,,
2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-17,1.0,2.0,5.0,3.0,NaN,4.0,7.0,6.0
2023-05-24,6.0,7.0,8.0,4.0,3.0,2.0,1.0,5.0
2023-05-31,5.0,2.0,3.0,NaN,NaN,NaN,1.0,4.0
2023-06-28,NaN,NaN,3.0,2.0,5.0,NaN,1.0,4.0
2023-07-05,4.0,5.0,NaN,7.0,6.0,3.0,2.0,1.0
2023-07-19,4.0,2.0,3.0,6.0,1.0,5.0,NaN,NaN


### improvement points

this is determined by the round score being less than the expected stroke count. in those cases, we find the difference, and where it's greater than zero, we sum the improvement

also testing with strict ratings comparison for improvement

In [23]:
improvement = (ratings - ratings_ave)*(SCALING_FACTOR * (ratings_ave/1000)).round(2)
improvement = improvement.where(improvement > 0).fillna(0).round(2)
# improvement.sum()

### points per week

In [29]:
ranked

name,Chase Ramsey,Isaac Richmond,Joe Bos,Joshua Owens,Kat Owens,Kevin Richmond,Kyle P-H,Noah Mashni
date,,,,,,,,
2023-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-17,1.0,2.0,5.0,3.0,NaN,4.0,7.0,6.0
2023-05-24,6.0,7.0,8.0,4.0,3.0,2.0,1.0,5.0
2023-05-31,5.0,2.0,3.0,NaN,NaN,NaN,1.0,4.0
2023-06-28,NaN,NaN,3.0,2.0,5.0,NaN,1.0,4.0
2023-07-05,4.0,5.0,NaN,7.0,6.0,3.0,2.0,1.0
2023-07-19,4.0,2.0,3.0,6.0,1.0,5.0,NaN,NaN


In [24]:
total_points = ranked.sum() + improvement.sum() + attendance['attendance'] + ctplp['extra_points']
total_points.drop('the_field')

name
Chase Ramsey      38.57
Isaac Richmond    38.98
Joe Bos           48.90
Joshua Owens      57.92
Kat Owens         48.01
Kevin Richmond    37.41
Kyle P-H          62.30
Noah Mashni       51.49
dtype: float64